In [1]:
import numpy as np
import json
import pandas as pd
from transformers import pipeline
import os
import torch
import torchvision
from transformers import BertTokenizer, BertForSequenceClassification, AutoModelWithLMHead, PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from torchmetrics.text.rouge import ROUGEScore
from transformers import Trainer, TrainingArguments, PegasusForCausalLM
from transformers import AutoTokenizer, AutoModelWithLMHead
from sklearn.model_selection import train_test_split
import evaluate

In [2]:
# rouge = ROUGEScore()

In [166]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [265]:
data = pd.read_csv('train.csv')
df = pd.DataFrame(data)
df = df.drop(['Unnamed: 0'], axis=1)
data_test = pd.read_csv('test.csv')
df_test = pd.DataFrame(data_test)
df_test = df_test.drop(['Unnamed: 0'], axis=1)

In [257]:
df_test.head()

,id,summary,stock,date
0,1,Canadian equities resumed their downward trend...,SHOP,2022-07-06
1,2,Destiny Wealth Partners LLC bought a new stake...,CARR,2022-07-03
2,3,"East West Bancorp, Inc. (NASDAQ:EWBC – ) – Inv...",EWBC,2022-07-25
3,4,Community Trust & Investment Co. increased its...,APH,2022-07-06
4,5,Valley Brook Capital Group Inc. lessened its s...,TJX,2022-07-19


In [ ]:
#Model for summarization
tokenizer1 = AutoTokenizer.from_pretrained('t5-base')
model1 = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)
inputs = tokenizer1.encode("summarize: " + article, return_tensors='pt', max_length=80, truncation=True)
summary_ids = model1.generate(inputs, max_length=80, min_length=80, length_penalty=5., num_beams=2)
summary = tokenizer1.decode(summary_ids[0])

In [236]:
# Model for stock Movement prediction
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finpred = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

loading configuration file https://huggingface.co/yiyanghkust/finbert-tone/resolve/main/config.json from cache at C:\Users\Alakh Agrawal/.cache\huggingface\transformers\70d1215b8fdafe3265e07d33b038971980e2f845842193d327e57e391fc58393.e033c3bdc62e71f1b657c29753d6b96d9d23207bef18e9ab1e2b6310e55d94e4
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Neutral",
    "1": "Positive",
    "2": "Negative"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 2,
    "Neutral": 0,
    "Positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab

In [266]:
df_pred = df_test
df_pred.tail()
df_test

,id,article,stock,date
0,1,Canadian equities resumed their downward trend...,SHOP,2022-07-06
1,2,Destiny Wealth Partners LLC bought a new stake...,CARR,2022-07-03
2,3,"East West Bancorp, Inc. (NASDAQ:EWBC – ) – Inv...",EWBC,2022-07-25
3,4,Community Trust & Investment Co. increased its...,APH,2022-07-06
4,5,Valley Brook Capital Group Inc. lessened its s...,TJX,2022-07-19
...,...,...,...,...
2495,2496,Comerica Bank increased its stake in Goldman S...,CMA,2022-07-29
2496,2497,USHG Acquisition Corp. (NYSE:HUGS – Get Rating...,HUGS,2022-07-17
2497,2498,"Advanced Micro Devices, Inc. (AMD) receives an...",AMD,2022-07-05
2498,2499,Sierra Capital LLC cut its holdings in Bristol...,BMY,2022-07-24


In [ ]:
for i in range(5):
    article = df_test['article'][i]
    inputs = tokenizer1.encode(article, return_tensors='pt', max_length=80, truncation=True)
    summary_ids = model1.generate(inputs, max_length=80, min_length=80, length_penalty=5., num_beams=2)
    summary = tokenizer1.decode(summary_ids[0])
    df_pred['article'][i] = summary

In [263]:
# df_pred.rename(columns = {'article':'summary'}, inplace = True)

In [ ]:
for i in range(len(df_pred['id'])):
    summary = df_pred['summary'][i]
    if finpred(summary)[1]=='Positive':
        

In [168]:
# article_list = []
# summ_list = []
# for i in range(len(df['article'])):
#     article_list.append(df['article'][i])
#     summ_list.append(df['summary'][i])

In [235]:
# inputs2 = summ_list[:3]
# summ_pred = finpred(inputs2)
# summ_pred
# print(finpred(yts))
# print(finpred(summary))

[{'label': 'Negative', 'score': 0.999864935874939}]
[{'label': 'Negative', 'score': 0.9999526739120483}]


In [16]:
#Fine tuning the model
X = df['article']
Y = df['summary']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [100]:
# from transformers import PegasusTokenizer

# Initialize the tokenizer
# tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-cnn')

# Define the train dataset as a list of text strings
train_texts = x_train[:100]
summary_text = y_train[:100]

# Convert the train dataset to the format required by the model
smaller_train_dataset = []
for text in train_texts:
    input_ids = tokenizer_summ.encode(text, return_tensors='pt', padding=True, truncation=True, max_length=50, add_special_tokens = True)
    attention_mask = [1] * len(input_ids)

    smaller_train_dataset.append({
        'input_ids': input_ids.flatten().tolist(),
        'attention_mask': attention_mask,
        'summary_text': summary_text
    })

In [232]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=32,  # Batch size
    save_steps=5000,                 # Save checkpoint every X steps
    save_total_limit=2,              # Keep only the last N checkpoints
)

# Fine-tune the model on the train dataset
# model_summ.train()
# model_summ.zero_grad()
# model_summ.fit(train_dataset, args=training_args)
train = Trainer(model=model1, tokenizer = tokenizer1, train_dataset = smaller_train_dataset, args=training_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [230]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=summary, references=yts)
results

ValueError: Mismatch in the number of predictions (297) and references (80)

In [94]:
# from sklearn.preprocessing import LabelEncoder as le
# # x_test1 = x_test
# # x_test1.dtype
# # x_test_smaller = x_test[1]
# xts2 = le.fit_transform(df['summary'][3],[xts])

In [233]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer1 = AutoTokenizer.from_pretrained('t5-base')
model1 = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)
inputs = tokenizer1.encode("summarize: " + article, return_tensors='pt', max_length=80, truncation=True)
summary_ids = model1.generate(inputs, max_length=80, min_length=80, length_penalty=5., num_beams=2)
summary = tokenizer1.decode(summary_ids[0])

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at C:\Users\Alakh Agrawal/.cache\huggingface\transformers\91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attentio

In [234]:
summary

'<pad> FTAC Athena Acquisition Corp. (NASDAQ:FTAAU) saw a significant drop in short interest in the month of October. as of October 15th, there was short interest totalling 100 shares, a drop of 94.7% from the September 30th total of 1,900 shares. based on an average daily volume of 43,700 shares,'